In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
from datetime import datetime

In [9]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [10]:
df = pd.read_csv("train.csv")
df.head(10)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,21 Lakh,17.50
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,NaN,5.20
7,9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,NaN,1.95
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,Petrol,Manual,First,21.56 kmpl,1462 CC,103.25 bhp,5.0,10.65 Lakh,9.95
9,11,Honda City 1.5 V AT Sunroof,Kolkata,2012,60000,Petrol,Automatic,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,NaN,4.49


## Part - a, b, c

## Eliminating CC from the values of Engine column

In [11]:
df['Engine'].isnull().sum()

np.int64(36)

In [12]:
df['Engine']=df['Engine'].apply(lambda x: x.split(' CC')[0] if isinstance(x, str) else x)

# Using Fuel_Type and Transmission filling null values or Engine

In [13]:
def fill_mode(group):
    mode_val = group.mode()
    if not mode_val.empty:
        return group.fillna(mode_val.iloc[0])
    else:
        return group

df['Engine'] = df.groupby(['Fuel_Type','Transmission'])['Engine'].transform(fill_mode)

In [14]:
df['Engine'].isnull().sum()

np.int64(0)

In [15]:
df['Power'].isnull().sum()

np.int64(36)

## Eliminating bhp from the values of Power column

In [16]:
df['Power']=df['Power'].apply(lambda x: x.split(' bhp')[0] if isinstance(x,str) else x)

# Using Fuel_Type and Transmission filling null values for Power column

In [17]:
def fill_mode(group):
    mode_val = group.mode()
    if not mode_val.empty:
        return group.fillna(mode_val.iloc[0])
    else:
        return group

df['Power'] = df.groupby(['Fuel_Type','Transmission'])['Power'].transform(fill_mode)

In [18]:
df['Power'].isnull().sum()

np.int64(0)

In [19]:
df['Seats'].isnull().sum()

np.int64(38)

## Using Engine and Power filling null values for Seats

In [20]:
def fill_mode(group):
    mode_val = group.mode()
    if not mode_val.empty:
        return group.fillna(mode_val.iloc[0])
    else:
        return group

df['Seats'] = df.groupby(['Engine','Power'])['Seats'].transform(fill_mode)

In [21]:
df['Seats'].isnull().sum()

np.int64(8)

## Dropping leftover null values of Seats that are filled using Engine & Power

In [22]:
df.dropna(subset=['Seats'],inplace=True)

In [23]:
df['New_Price'].isnull().sum()

np.int64(5024)

## Removing Lakh from the values of New_Price column

In [24]:
df['New_Price']=df['New_Price'].apply(lambda x: x.split('Lakh')[0] if isinstance(x,str) else x)

### Filling the null values of New_Price by using Car's Make as most of the values are null and other combinations of columns are not working for filling the null values of New_Price column

In [25]:
def fill_mode(group):
    mode_val = group.mode()
    if not mode_val.empty:
        return group.fillna(mode_val.iloc[0])
    else:
        return group

df['New_Price'] = df.groupby(['Name'])['New_Price'].transform(fill_mode)

In [26]:
df['New_Price'].isnull().sum()

np.int64(4837)

### Filling remaining values using the mean of Proce column

In [27]:
df['New_Price']=df.groupby(['Name','Year','Fuel_Type','Transmission'])['New_Price'].transform(lambda x: x.fillna(df['Price'].mean()))

<ipython-input-27-9acfd775dfb5>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['New_Price']=df.groupby(['Name','Year','Fuel_Type','Transmission'])['New_Price'].transform(lambda x: x.fillna(df['Price'].mean()))


In [28]:
df['New_Price'].isnull().sum()

np.int64(0)

### Removing kmpl from the values of Mileage column

In [29]:
df['Mileage']=df['Mileage'].apply(lambda x:x.split('kmpl')[0] if isinstance(x,str) else x)

In [30]:
df['Mileage'].isnull().sum()

np.int64(2)

### Dropping null values of Mileage column

In [32]:
df=df.dropna(subset=['Mileage'])

In [33]:
df['Mileage'].isnull().sum()

np.int64(0)

### Encoding the values of Fuel_Type and Transmission columns

In [34]:
df_encoded = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

In [35]:
df_encoded.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.2,5.0,9.644818,12.50,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13 km/kg,1199,88.7,5.0,8.61,4.50,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7.0,11.27,6.00,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2,1968,140.8,5.0,9.644818,17.74,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.1,5.0,9.644818,3.50,False,True


### (d) Creating a new column Car_Age for calculating the car's age

In [38]:
current_year = datetime.now().year
df['CarAge'] = current_year-df['Year']

In [39]:
df.head(5)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,CarAge
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.2,5.0,9.644818,12.50,10
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199,88.7,5.0,8.61,4.50,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,11.27,6.00,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2,1968,140.8,5.0,9.644818,17.74,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.1,5.0,9.644818,3.50,12


## (e)

In [40]:
selected_df = df[['Name', 'Year', 'Price']]

In [41]:
filtered_df = df[df['Fuel_Type'] == 'Diesel']

In [42]:
renamed_df = df.rename(columns={'Kilometers_Driven': 'KMs_Driven', 'Fuel_Type': 'Fuel'})

In [43]:
from datetime import datetime
df['Car_Age'] = datetime.now().year - df['Year']

In [44]:
arranged_df = df.sort_values(by='Price', ascending=False)

In [45]:
summary_df = df.groupby('Fuel_Type')['Price'].mean().reset_index()